In [10]:
from sentence_transformers import SentenceTransformer
import faiss
import pickle
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, SystemMessage, AIMessage
import os

In [43]:
class QueryService:
    def __init__(self, index):
        self.index = index  
        self.model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  
    
    def retrieve(self, query, k=5):
        query_vector = self.model.encode([query])
        D, I = self.index.search(query_vector, k)        
        return I, D
    
    def make_context(self, indexes, chunks):
        relevant_chunks = []
        for row in indexes:
            for i in row:
                relevant_chunks.append(chunks[i])

        context = " ".join(relevant_chunks)
        return context
    
class DataLoader:
    def __init__(self):
        pass

    def load_chunks(self):
        with open('chunks.pkl', 'rb') as f:
            chunks = pickle.load(f)

        return chunks
    
    def load_faiss_index(self):
        return faiss.read_index('faiss_index.index')
    
class Model():
    def __init__(self):
        prompt_template = ChatPromptTemplate.from_messages([
            MessagesPlaceholder("user_query"),
            MessagesPlaceholder("context")
        ])
        llm = ChatOpenAI(model_name="gpt-3.5-turbo")
        self.chain = LLMChain(llm=llm, prompt=prompt_template)

    def answer(self, query, context):
        query = [HumanMessage(content=query)]
        context = [SystemMessage(content=context)]

        response = self.chain.run({
            "user_query": query,  
            "context": context  
        })
        
        return response


In [44]:
os.environ["OPENAI_API_KEY"] = ""

dataLoader = DataLoader()
chunks = dataLoader.load_chunks()
index = dataLoader.load_faiss_index()
queryService = QueryService(index)

user_query = 'Why space mission data is not immediately available?'

I, D = queryService.retrieve(user_query)

context = queryService.make_context(I, chunks)

In [45]:
model = Model()
response = model.answer(user_query, context)
print(response)

Space mission data is not immediately available for several reasons:

1. Data processing: Raw data collected from space missions needs to be processed, analyzed, and verified before it can be released to the public. This process can take time to ensure accuracy and reliability.

2. Communication delays: Space missions often involve communication with spacecraft that are far away from Earth. Data must be transmitted back to Earth, which can take time due to the vast distances involved.

3. Security and confidentiality: Some space mission data may be sensitive or classified, requiring careful review and approval before it can be released to the public.

4. Collaboration and coordination: Space missions are often conducted by international teams or involve multiple organizations working together. Data sharing agreements and coordination efforts may need to be finalized before data can be made public.

5. Quality control: Space agencies and research institutions have strict quality control